# NLP From Scratch 

|  | Tuesday 4-5:15pm | Friday  4-5:30pm |
|:------:|:-------------------------------------------:|:--------------------------------------------------------------------------:|
| **Week 1** | Introduction | Introduction |
| **Week 2** | Custom computer vision tasks | State of the art in Computer Vision |
| **Week 3** | Introduction to Tabular modeling and pandas | Pandas workshop and feature engineering |
| **Week 4** | Tabular and Image Regression | Feature importance and advanced feature  engineering |
| **Week 5** | Natural Language Processing | State of the art in NLP |
| **Week 6** | Segmentation and Kaggle | Audio |
| **Week 7** | Computer vision from scratch | **NLP from scratch** |
| **Week 8** | Callbacks | Optimizers |
| **Week 9** | Generative Adversarial Networks | Research time / presentations |
| **Week 10** | Putting models into production | Putting models into production |


This week we will focus on building custom models for both a text and an image problem. We will explore Recurrent Nerual Networks and Convolutional Neural Networks, and how to give a model the ability to have 'memory'

## What is a Recursive Neural Network (RNN)?

There is a large difference between RNN versus a CNN. To quote Andrej Karpathy, "they (CNN's) accept a fixed-sized vector as input (e.g. an image) and produce a fixed-sized vector as output (e.g. probabilities of different classes). Not only that: These models perform this mapping using a fixed amount of computational steps (e.g. the number of layers in the model). The core reason that recurrent nets are more exciting is that they allow us to operate over sequences of vectors: Sequences in the input, the output, or in the most general case both.  "

![alt text](http://karpathy.github.io/assets/rnn/diags.jpeg)

*key:* 
* Red: input vector (tensor)
* Green and Blue: Output vectors that hold the state (more on this later)
* Arrows: Functions (eg. Matrix Multiplication)

## When to use them?

* They work best with sequences (stock data, NLP)

## Why use them?

* They are flexible and work with many structures:
  * **One-to-many** Take an image and return a caption
  * **Many-to-one** Take a movie review and return if it's positive or negative
  * **Many-to-Many** Take an English sentence and return a Spanish sentence

## Human Numbers

We will explore a basic Recursive Neural Network and try to apply it to numbers written in text form, as well as get a look at custom model architecture for fully-connected models

In [0]:
from fastai import *
from fastai.text import *
bs=64

# Data

In [0]:
path = untar_data(URLs.HUMAN_NUMBERS)
path.ls()

[PosixPath('/root/.fastai/data/human_numbers/train.txt'),
 PosixPath('/root/.fastai/data/human_numbers/valid.txt')]

In [0]:
open(path/'train.txt').readlines()[:5]

['one \n', 'two \n', 'three \n', 'four \n', 'five \n']

In [0]:
def readnums(d): return [', '.join(o.strip() for o in open(path/d).readlines())]
# Strip: returns a copy of a string with both leading and trailing characters stripped

In [0]:
train_txt = readnums('train.txt'); train_txt[0][-80:]

'even thousand nine hundred ninety eight, seven thousand nine hundred ninety nine'

In [0]:
valid_txt = readnums('valid.txt'); valid_txt[0][-80:]

' nine thousand nine hundred ninety eight, nine thousand nine hundred ninety nine'

In [0]:
train_txt[0][0:5]

'one, '

## ItemList

Part of the Fast.AI framework consists of ItemLists. They are everywhere and are used for all applications we want to dictate: 

https://docs.fast.ai/tutorial.itemlist.html



In [0]:
TextList??

In [0]:
ItemList??

We've already been playing with them

In [0]:
TabularList.from_df()
ImageList.from_folder()

The source code:

In [0]:
class ImageDataBunch(DataBunch):
  'Databunch suitable for computer vision.'
  
  @classmethod
  def from_folder(cls, path:PathorStr, train:PathOrStr='train', valid:PathOrStr='valid',
                    valid_pct=None, seed:int=None, classes:Collection=None, **kwargs:Any)->'ImageDataBunch':
    'Create from imagenet style dataset in `path` with `train`, `valid`, `test` subfolders (or provide `valid_pct`)'
    path = Path(path)
    il = ImageList.from_folder(path)
    if valid_pct is None: src = il.split_by_folder(train=train, valid=valid)
    else: src = il.split_by_rand_pct(valid_pct, seed)
    src = src.label_from_folder(classes=classes)
    return cls.create_from_ll(src, **kwargs)

In [0]:
class TabularList(ItemList):
    "Basic `ItemList` for tabular data."
    _item_cls=TabularLine
    _processor=TabularProcessor
    _bunch=TabularDataBunch
    def __init__(self, items:Iterator, cat_names:OptStrList=None, cont_names:OptStrList=None,
                 procs=None, **kwargs)->'TabularList':
        super().__init__(range_of(items), **kwargs)
        #dataframe is in inner_df, items is just a range of index
        if cat_names is None:  cat_names = []
        if cont_names is None: cont_names = []
        self.cat_names,self.cont_names,self.procs = cat_names,cont_names,procs
        self.copy_new += ['cat_names', 'cont_names', 'procs']
        self.preprocessed = False
        
    @class method
     def from_df(cls, df:DataFrame, cat_names:OptStrList=None, cont_names:OptStrList=None, procs=None, **kwargs)->'ItemList':
        "Get the list of inputs in the `col` of `path/csv_name`."
        return cls(items=range(len(df)), cat_names=cat_names, cont_names=cont_names, procs=procs, inner_df=df.copy(), **kwargs)

The steps for making a databunch also are as follows: ItemList, split the list, label the lists, seperate into batches and databunch

<h3>self vs cls:</h3>

`self` is used for the first argument in *instance* methods

`cls` is used for the first argument in *class* methods

Today we will use **TextList**

In [0]:
train = TextList(train_txt, path=path)
valid = TextList(valid_txt, path=path)

In [0]:
src = ItemLists(path=path, train=train, valid=valid).label_for_lm()

In [0]:
data = src.databunch(bs=bs)

In [0]:
data

In [0]:
len(data.valid_ds[0][0].data)

13017

In [0]:
data.bptt

70

BBPT = Backpropogation Through Time, used for Recurrent Nueral Networks: https://en.wikipedia.org/wiki/Backpropagation_through_time

In [0]:
data.bptt, len(data.valid_dl)

(70, 3)

In [0]:
13017/70/bs

2.905580357142857

In [0]:
it = iter(data.valid_dl); it

<generator object DeviceDataLoader.__iter__ at 0x7f5cf273eca8>

In [0]:
x1, y1 = next(it)
x2, y2 = next(it)
x3, y3 = next(it)
it.close()

`numel` tells us how many elements are in a PyTorch tensor

In [0]:
x1.numel()

4480

In [0]:
x1.shape, 16*70

(torch.Size([64, 70]), 1120)

In [0]:
x1.shape, y1.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [0]:
x1[:,0]

tensor([ 2,  9, 11, 12, 13, 11, 10,  9, 10, 14, 19, 25, 19, 15, 16, 11, 19,  9,
        10,  9, 19, 25, 19, 11, 19, 11, 10,  9, 19, 20, 11, 26, 20, 23, 20, 20,
        24, 20, 11, 14, 11, 11,  9, 14,  9, 20, 10, 20, 35, 17, 11, 10,  9, 17,
         9, 20, 10, 20, 11, 20, 11, 20, 20, 20], device='cuda:0')

In [0]:
y1[:,0]

tensor([19, 19, 27, 10,  9, 12, 32, 19, 26, 10, 11, 15, 11, 10,  9, 15, 11, 19,
        26, 19, 11, 18, 11, 18,  9, 18, 21, 19, 10, 10, 20,  9, 11, 16, 11, 11,
        13, 11, 13,  9, 13, 14, 20, 10, 20, 11, 24, 11,  9,  9, 16, 17, 20, 10,
        20, 11, 24, 11, 19,  9, 19, 11, 11, 10], device='cuda:0')

In [0]:
data.valid_ds.vocab

In [0]:
v = data.valid_ds.vocab

In [0]:
x1[0]

tensor([ 2, 19, 11, 12,  9, 19, 11, 13,  9, 19, 11, 14,  9, 19, 11, 15,  9, 19,
        11, 16,  9, 19, 11, 17,  9, 19, 11, 18,  9, 19, 11, 19,  9, 19, 11, 20,
         9, 19, 11, 29,  9, 19, 11, 30,  9, 19, 11, 31,  9, 19, 11, 32,  9, 19,
        11, 33,  9, 19, 11, 34,  9, 19, 11, 35,  9, 19, 11, 36,  9, 19],
       device='cuda:0')

In [0]:
v.textify(x1[0])

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight'

In [0]:
v.textify(y1[0])

'eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand'

In [0]:
v.textify(x2[0])

'thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two ,'

In [0]:
v.textify(y2[0])

'eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two , eight'

In [0]:
data.show_batch(ds_type=DatasetType.Valid)

idx,text
0,"thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand sixty , eight thousand sixty"
1,"eight , eight thousand eighty nine , eight thousand ninety , eight thousand ninety one , eight thousand ninety two , eight thousand ninety three , eight thousand ninety four , eight thousand ninety five , eight thousand ninety six , eight thousand ninety seven , eight thousand ninety eight , eight thousand ninety nine , eight thousand one hundred , eight thousand one hundred one , eight thousand one"
2,"thousand one hundred twenty four , eight thousand one hundred twenty five , eight thousand one hundred twenty six , eight thousand one hundred twenty seven , eight thousand one hundred twenty eight , eight thousand one hundred twenty nine , eight thousand one hundred thirty , eight thousand one hundred thirty one , eight thousand one hundred thirty two , eight thousand one hundred thirty three , eight thousand"
3,"three , eight thousand one hundred fifty four , eight thousand one hundred fifty five , eight thousand one hundred fifty six , eight thousand one hundred fifty seven , eight thousand one hundred fifty eight , eight thousand one hundred fifty nine , eight thousand one hundred sixty , eight thousand one hundred sixty one , eight thousand one hundred sixty two , eight thousand one hundred sixty three"
4,"thousand one hundred eighty three , eight thousand one hundred eighty four , eight thousand one hundred eighty five , eight thousand one hundred eighty six , eight thousand one hundred eighty seven , eight thousand one hundred eighty eight , eight thousand one hundred eighty nine , eight thousand one hundred ninety , eight thousand one hundred ninety one , eight thousand one hundred ninety two , eight thousand"


# Introduction into Models

Models are split into two parts: **__init__**, and **__forward__** and are of type **nn.Module**

In [0]:
nn.Module??

In [0]:
class model(nn.Module):
  def __init__(self):
    super().__init__() # Compatability helper
    self.l1 = nn.Linear(nv, nh)
    self.l2 = nn.Linear(nh, 10)

In [0]:
model()

model(
  (l1): Linear(in_features=40, out_features=64, bias=True)
  (l2): Linear(in_features=64, out_features=10, bias=True)
)

**init** builds the *structure*, **forward** is the *functionality*

In [0]:
class model(nn.Module):
  def __init__(self):
    super().__init__() # Compatability helper
    self.l1 = nn.Linear(nv, nh)
    self.l2 = nn.Linear(nh, 10)
  def forward(self, x):
    return(self.l2(self.l1(x)))

In [0]:
model().forward(Tensor(40))

tensor([ 0.0382,  0.0881,  0.0685, -0.0065, -0.0288, -0.1310,  0.1076, -0.0007,
         0.1658,  0.0477], grad_fn=<AddBackward0>)

How does it change?

In [0]:
class model(nn.Module):
  def __init__(self):
    super().__init__() # Compatability helper
    self.l1 = nn.Linear(nv, 10)
  def forward(self, x):
    return(self.l1(x))

In [0]:
model().forward(Tensor(40))

tensor([ 0.1197,  0.0008,  0.1412,  0.0399, -0.0981, -0.0166, -0.0311,  0.0912,
        -0.1451, -0.0644], grad_fn=<AddBackward0>)

# A Basic Fully Connected Model

A convolutional model is only connected to the next few layers, wheras a fully connected model, each layer is connected. 'Memory'.

In [0]:
data = src.databunch(bs=bs, bptt=3)

In [0]:
x,y = data.one_batch()
x.shape, y.shape

(torch.Size([64, 3]), torch.Size([64, 3]))

Let `nv` to be inputs, and `nh` to be the size of our hidden layer

In [0]:
nv = len(v.itos); nv

40

In [0]:
nh = 64

Let's make a loss function and metric

In [0]:
def loss4(input, targ): return F.cross_entropy(input, targ[:,-1])
def acc4(input, targ): return accuracy(input, targ[:,-1])

And now our model, which will predict word 4 using words 1, 2, and 3

![alt text](https://github.com/hiromis/notes/raw/master/lesson7/49.png)

In [0]:
nv, nh

(40, 64)

In [0]:
class Model0(nn.Module):
  def __init__(self):
    super().__init__()
    self.i_h = nn.Embedding(nv, nh) # <----- 1, green arrow
    self.h_h = nn.Linear(nh, nh)    # <----- 2, orange arrow
    self.h_o = nn.Linear(nh,nv)     # <----- 3, blue arrow
    self.bn = nn.BatchNorm1d(nh)    # <----- 4
    
  def forward(self, x):
    h = self.bn(F.relu(self.h_h(self.i_h(x[:,0])))) # <---- 1, 2, 4
    if x.shape[0]>1:
      h += self.i_h(x[:,1])               # <---- 1
      h = self.bn(F.relu(self.h_h(h)))  # <---- 2, 4
    if x.shape[0]>2:
      h += self.i_h(x[:,2])               # <---- 1
      h = self.bn(F.relu(self.h_h(h)))  # <---- 2, 4
    return self.h_o(h)                  # <---- 3

In [0]:
Model0()

Model0(
  (i_h): Embedding(40, 64)
  (h_h): Linear(in_features=64, out_features=64, bias=True)
  (h_o): Linear(in_features=64, out_features=40, bias=True)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [0]:
learn = Learner(data, Model0(), loss_func=loss4, metrics=acc4)

In [0]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.584361,3.643630,0.097426,00:01
1,3.066797,3.174392,0.339154,00:01
2,2.518950,2.691588,0.448759,00:01
3,2.197248,2.422778,0.454274,00:01
4,2.065464,2.328205,0.457950,00:01
5,2.037472,2.315218,0.458180,00:01


## Same mode, but with a loop

In [0]:
class Model1(nn.Module):
  def __init__(self):
    super().__init__()
    self.i_h = nn.Embedding(nv, nh) # <----- 1, green arrow
    self.h_h = nn.Linear(nh, nh)    # <----- 2, orange arrow
    self.h_o = nn.Linear(nh,nv)     # <----- 3, blue arrow
    self.bn = nn.BatchNorm1d(nh)    # <----- 4
    
  def forward(self, x):
    h = torch.zeros(x.shape[0], nh).to(device=x.device)
    for i in range(x.shape[1]):
      h += self.i_h(x[:,i])
      h = self.bn(F.relu(self.h_h(h)))
    return self.h_o(h)

In [0]:
learn = Learner(data, Model1(), loss_func=loss4, metrics=acc4)

In [0]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.610184,3.554779,0.094899,00:01
1,3.146208,3.096071,0.338465,00:01
2,2.598369,2.626821,0.379596,00:01
3,2.259510,2.363997,0.417279,00:01
4,2.121799,2.263034,0.434053,00:01
5,2.092541,2.247800,0.434743,00:01


## Multi-fully connected model

In [0]:
data = src.databunch(bs=bs, bptt=20)

In [0]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 20]), torch.Size([64, 20]))

In [0]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        res = []
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.h_o(self.bn(h)))
        return torch.stack(res, dim=1)

In [0]:
learn = Learner(data, Model2(), metrics=accuracy)

In [0]:
learn.fit_one_cycle(10, 1e-4, pct_start=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,3.729914,3.671241,0.038636,00:00
1,3.619103,3.557339,0.084162,00:00
2,3.479878,3.441949,0.168608,00:00
3,3.335397,3.338587,0.235014,00:00
4,3.203775,3.256569,0.279190,00:00
5,3.095800,3.195556,0.343111,00:00
6,3.014856,3.155470,0.378764,00:00
7,2.959642,3.133752,0.399787,00:00
8,2.926201,3.125398,0.405966,00:00
9,2.908853,3.124154,0.406889,00:00


## Maintain the current state (start of recurrance/memory)

In [0]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(bs, nh).cuda()
        
    def forward(self, x):
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        res = self.h_o(res)
        return res

In [0]:
learn = Learner(data, Model3(), metrics=accuracy)

In [0]:
learn.model

Model3(
  (i_h): Embedding(40, 64)
  (h_h): Linear(in_features=64, out_features=64, bias=True)
  (h_o): Linear(in_features=64, out_features=40, bias=True)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [0]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.625595,3.501460,0.062713,00:00
1,3.276977,2.903179,0.401776,00:00
2,2.611589,2.015941,0.451349,00:00
3,2.016599,1.813454,0.390128,00:00
4,1.674391,1.760234,0.466335,00:00
5,1.459975,1.802815,0.487642,00:00
6,1.294104,1.805581,0.505753,00:00
7,1.144800,1.690800,0.527131,00:00
8,0.999956,1.662817,0.552060,00:00
9,0.864257,1.473552,0.578622,00:00


## nn.RNN


"Explaining the Vanishing Gradient Problem
As you remember, the gradient descent algorithm finds the global minimum of the cost function that is going to be an optimal setup for the network.

As you might also recall, information travels through the neural network from input neurons to the output neurons, while the error is calculated and propagated back through the network to update the weights.

It works quite similarly for RNNs, but here we’ve got a little bit more going on.

Firstly, information travels through time in RNNs, which means that information from previous time points is used as input for the next time points.
Secondly, you can calculate the cost function, or your error, at each time point.

Basically, during the training, your cost function compares your outcomes (red circles on the image below) to your desired output.

As a result, you have these values throughout the time series, for every single one of these red circles."

https://www.superdatascience.com/blogs/recurrent-neural-networks-rnn-the-vanishing-gradient-problem/
![alt text](https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/31_blog_image_1.png) 
![alt text](https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/31_blog_image_2.png)



Essentially, we are calculating so many gradients at one given time that either if there are too many (think we are dealing with a paragraph of text at a time) we explode.

Too small or too few, it simply vainishes

In [0]:
nn.RNN??

In [0]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.RNN(nh,nh, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(1, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [0]:
Model4()

Model4(
  (i_h): Embedding(40, 64)
  (rnn): RNN(64, 64, batch_first=True)
  (h_o): Linear(in_features=64, out_features=40, bias=True)
  (bn): BatchNorm1dFlat(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [0]:
learn = Learner(data, Model4(), metrics=accuracy)

In [0]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.613025,3.509639,0.220526,00:00
1,3.196244,2.706012,0.460938,00:00
2,2.503634,2.037579,0.444886,00:00
3,1.944358,1.839663,0.463139,00:00
4,1.598980,1.753945,0.476207,00:00
5,1.316993,1.680247,0.474787,00:00
6,1.104018,1.586872,0.512855,00:00
7,0.949063,1.635032,0.539489,00:00
8,0.822824,1.636205,0.548224,00:00
9,0.720250,1.688482,0.563423,00:00


## GRU

Improved version of a standard RNN, which has a vanishing gradient issue, we use **update gates and reset gates**, which are two vectors that decide whether information should be passed to the output.

In [0]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh, nh, 2, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [0]:
Model5()

Model5(
  (i_h): Embedding(40, 64)
  (rnn): GRU(64, 64, num_layers=2, batch_first=True)
  (h_o): Linear(in_features=64, out_features=40, bias=True)
  (bn): BatchNorm1dFlat(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [0]:
learn = Learner(data, Model5(), metrics=accuracy)

In [0]:
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.964803,2.443228,0.455256,00:00
1,1.811684,1.526673,0.617827,00:00
2,0.945482,1.119839,0.793466,00:00
3,0.461102,1.088649,0.824432,00:00
4,0.230549,1.151060,0.828551,00:00
5,0.122105,1.052200,0.834588,00:00
6,0.067979,1.047116,0.836222,00:00
7,0.040585,1.099488,0.834801,00:00
8,0.025961,1.136753,0.834233,00:00
9,0.018535,1.131347,0.833594,00:00
